In [ ]:
from astropy.table import Table, join
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd


ForGroup = Table.read('../../GAMA_DATA/G3CFoFGroupv10.fits')

## Clean Data

binCount = 100

fig = plt.figure(figsize=(12, 16), dpi=300)

xfield = 'TotRmag'
ax1 = fig.add_subplot(1, 1, 1)
ax1.title.set_text('Group Counts versus '+xfield)
ax1.set_ylabel('Group Count')
ax1.set_xlabel(xfield)

an, bn =stats.norm.fit(ForGroup[xfield].data)
print(f"{xfield} Normal Fit {an}, {bn}")
xn0, xn1 = stats.norm.ppf([0.01, 0.99], an, scale = bn)
xn = np.linspace(xn0,xn1,100)

ag, bg, cg =stats.gamma.fit(ForGroup[xfield].data)
print(f"{xfield} Gamma fit : {ag} {bg} {cg}")
xg0, xg1 = stats.gamma.ppf([0.01, 0.99], ag, loc = bg, scale=cg)
xg = np.linspace(xg0,xg1,100)

aj, bj, cj, dj =stats.johnsonsu.fit(ForGroup[xfield].data)
print(f"{xfield} Johnson fit : {aj} {bj} {cj} {dj}")
xj0, xj1 = stats.johnsonsu.ppf([0.01, 0.99], aj, bj, loc=cj, scale=dj)
xj = np.linspace(xj0,xj1,100)

# Perform Cox Box
#print(stats.boxcox(RErange[xfield]))
transdata, lamda = stats.boxcox(-ForGroup[xfield])
print(f"{xfield} BoxCox lamda {lamda}")
acn ,bcn =stats.norm.fit(transdata)
print(f"{xfield} Normal fit : {acn} {bcn}")
xcn0, xcn1 = stats.norm.ppf([0.01, 0.99], acn, scale = bcn)
xcn = np.linspace(xcn0,xcn1,100)
# y values for CoxBox
ycn = (lamda / xcn**lamda-1)

ax1.hist(ForGroup[xfield].data, bins=binCount, density=True)
ax1.plot(xn, stats.norm.pdf(xn, an, bn),'y-', lw=2, alpha=0.6, label='normal pdf')
ax1.plot(xg, stats.gamma.pdf(xg, ag, loc=bg, scale=cg),'g-', lw=2, alpha=0.6, label='gamma pdf')
ax1.plot(xj, stats.johnsonsu.pdf(xj, aj, bj, loc=cj, scale=dj),'r-', lw=2, alpha=0.6, label='Johnson pdf')
ax1.legend()
fig.savefig('../../ChartsPlots/GroupTotRmagHistogram.png', dpi=300, bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(10, 8), dpi=300)


ForGroup = Table.read('../../GAMA_DATA/G3CFoFGroupv10.fits')
print(f"ForGroup : {len(ForGroup)}")
## Clean Data

GroupGal = Table.read('../../GAMA_DATA/G3CGalv10.fits')
# Clean data
print(f"Before Clean GroupGal : {len(GroupGal)}")
GroupGal = GroupGal[GroupGal['GroupID'] > 0 ]
print(f"After Clean GroupGal : {len(GroupGal)}")

StellarMasses = Table.read('../../GAMA_DATA/StellarMassesv19.fits')
# Clean Data
#StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['metal'] > 0.001]


StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['metal'] > 0.01]
print(f"StellarMasses : {len(StellarMasses)}")


envMeasures = Table.read('../../GAMA_DATA/EnvironmentMeasuresv05.fits')
print(f"EnvMeasures : {len(envMeasures)}")
# Clean Data
envMeasures = envMeasures[envMeasures['SurfaceDensity'] < 15]
envMeasures = envMeasures[envMeasures['AGEDenParFlag'] == 0]
envMeasures = envMeasures[envMeasures['CountInCylFlag'] == 0]
envMeasures = envMeasures[envMeasures['DistanceTo5nn'] > 0.1]
envMeasures = envMeasures[envMeasures['CountInCyl'] > 0]

visualMorph = Table.read('../../GAMA_Data/VisualMorphologyv03.fits')
print(f"visualMorph : {len(visualMorph)}")
## Clean Data
visualMorph = visualMorph[visualMorph['ELLIPTICAL_CODE'] == 1]

envClass = Table.read('../../GAMA_DATA/GalaxiesClassifiedv01.fits')
envClass = envClass['CATAID','GeoS4']

#print(f"Group Gal size before join : {len(GroupGal)}")
#GroupGal = join(GroupGal,ForGroup,keys='GroupID',join_type='inner')
#print(f"Group Gal size after Join : {len(GroupGal)}")

GroupGal = join(GroupGal,visualMorph,keys='CATAID',join_type='inner')

GroupGal = join(GroupGal,envClass,keys='CATAID',join_type='inner')
print('Joining Environment Classes '+str(len(GroupGal)))

GroupGal = join(GroupGal,envMeasures,keys='CATAID',join_type='inner')
print('Joining Environment Measures '+str(len(GroupGalMeasures)))
#print(FinalData.colnames)

#GroupGalStellar = join(GroupGal,StellarMasses,keys='CATAID',join_type='inner')
#print(f"Join GroupGal & StellarMasses {len(DataLocalGroup)}")
#print(DataLocalGroup.colnames)

DataLocalGroup = join(GroupGal,StellarMasses,keys='CATAID',join_type='inner')
print(f"Join GroupGal & StellarMasses {len(DataLocalGroup)}")
print('GroupGal & StellarMasses')
print(DataLocalGroup.colnames)

import math

# Sum logmstar over Local Group
DLG_Groups = DataLocalGroup.group_by('GroupID')

GroupLogStar = DLG_Groups['GroupID','logmstar'].groups.aggregate(lambda x: math.log10(np.sum(10**x)))
print("GroupLogStar")
print(GroupLogStar.colnames)

#LocalGroupMeasures = join(LocalGroupMeasures,envClass,keys='CATAID',join_type='inner')
#print('Joining Environment Measures '+str(len(LocalGroupMeasures)))
#print(FinalData.colnames)


RErange = join(ForGroup,GroupGal,keys='GroupID',join_type='inner')

yfield = 'TotRmag'
alphaVal = .3

fig = plt.figure(figsize=(11, 8), dpi=300)
#fig.suptitle('Density Plots - Group Redness (TotRmag) versus Environments Galaxies  for Elliptical Galaxies')
xfields = ['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar']     
y = RErange[yfield]
ymin = min(y)
print(ymin)
ymax = max(y)
print(ymax)
for i,xfield in enumerate(xfields,1) :
    x = RErange[xfield]
    xmin = min(x)
    xmax = max(x)
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    ax = fig.add_subplot(1, len(xfields), i)
    ax.set_ylabel(yfield)
    ax.set_xlabel(xfield)
    #ax.invert_yaxis()           
    values = np.vstack([x, y])
    kernel = stats.gaussian_kde(values)
    Z = np.reshape(kernel(positions).T, X.shape)
    #ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,extent=[xmin, xmax, ymin, ymax])
    #ax.imshow(np.rot90(Z), cmap='PuBu',extent=[xmin, xmax, ymin, ymax])
    #ax.imshow(np.rot90(Z), cmap='PuBu')
    ax.contour(X, Y, Z)
    #ax.plot(x, y, 'k.', markersize=2)
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    r, p = stats.pearsonr(x,y)
    print(f"Pearson Correlation x: {xfield} y: {yfield} Correlation : {r} T-Test : {p}")
    r, p = stats.spearmanr(x,y)
    print(f"Spearman Correlation x: {xfield} y: {yfield} Correlation : {r} T-Test : {p}")
    m, c, r, p, se = stats.linregress(x,y)
    print(f"Linear Regresion x: {xfield} y: {yfield} slope : {m} Intercept {c} Correlation {r} Wald test {p} {se}")
plt.show()                            
#fig.legend(loc="upper right")
#plt.axis([0, 3, 0, 0.8])
fig.tight_layout()
#fig.savefig('../../ChartsPlots/GroupDensityTotRmagEnvironments.png', dpi=300, bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(11, 4), dpi=300)
#fig.suptitle('Density Plots - Group Redness (TotRmag) versus Log(Environments) Galaxies  for Elliptical Galaxies')
xfields = ['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar']
i = 1      
y = RErange[yfield]
ymin = min(y)
print(ymin)
ymax = max(y)
print(ymax)
for xfield in xfields :
    x = np.log10(RErange[xfield]+0.01)
    xmin = min(x)
    xmax = max(x)
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    ax = fig.add_subplot(1, len(xfields), i)
    ax.set_ylabel(yfield)
    ax.set_xlabel('log10('+xfield+')')
    #ax.invert_yaxis()
    i += 1           
    values = np.vstack([x, y])
    kernel = stats.gaussian_kde(values)
    Z = np.reshape(kernel(positions).T, X.shape)
    #ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,extent=[xmin, xmax, ymin, ymax])
    #ax.imshow(np.rot90(Z), cmap='PuBu',extent=[xmin, xmax, ymin, ymax])
    #ax.imshow(np.rot90(Z), cmap='PuBu')
    ax.contour(X, Y, Z)
    #ax.plot(x, y, 'k.', markersize=2)
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    r, p = stats.pearsonr(x,y)
    print(f"Log - Pearson Correlation x: {xfield} y: {yfield} Correlation : {r} T-Test : {p}")
    r, p = stats.spearmanr(x,y)
    print(f"Log - Spearman Correlation x: {xfield} y: {yfield} Correlation : {r} T-Test : {p}")
    m, c, r, p, se = stats.linregress(x,y)
    print(f"Log - Linear Regresion x: {xfield} y: {yfield} slope : {m} Intercept {c} Correlation {r} Wald test {p} {se}")
    
plt.show()                            
fig.tight_layout()
#fig.savefig('../../ChartsPlots/GroupDensityTotRmagLogEnvironments.png', dpi=300, bbox_inches='tight')
plt.show()



GroupGal = GroupGal.to_pandas()
print(GroupGal.columns)
#GroupGalMeasures['CountInCyl']    = np.log10(GroupGalMeasures['CountInCyl'])
print(min(GroupGal['DistanceTo5nn']))
print(max(GroupGal['DistanceTo5nn']))
#GroupGalMeasures['DistanceTo5nn'] = np.log10(GroupGalMeasures['DistanceTo5nn'])
#GroupGalMeasures['SurfaceDensity'] = np.log10(GroupGalMeasures['SurfaceDensity'])
#GroupGalMeasures['AGEDenPar'] = np.log10(GroupGalMeasures['AGEDenPar'])

GroupEnv = GroupGal.groupby('GroupID')['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'].agg(np.mean)
print("GroupEnv")
print(GroupEnv.columns)

ForGroup = ForGroup.to_pandas()
#GroupData = join(ForGroup,GroupLog,keys='GroupID',join_type='inner')
#print(f"Group Gal size after Join : {len(GroupGal)}")

GroupData = ForGroup.merge(GroupEnv, left_on=('GroupID'), right_on=('GroupID'))
print(GroupData.columns)

#fig = plt.figure(figsize=(10, 8), dpi=300)
fig, axes = plt.subplots(4,4,figsize=(12, 12), sharey=True) 
#fig.suptitle('Countour Plots - Group Redness versus Log(Environments) for Galaxy Type')

types = [(0,'Voids'),(1,'Sheet'),(2,'Filament'),(3,'Knot')]
xfields = ['CountInCyl_y','DistanceTo5nn_y','SurfaceDensity_y','AGEDenPar_y']
i = 0
j = 0

for j, gt in enumerate(types):
    prerange = GroupGal[GroupGal['GeoS4'] == gt[0]]
    subrange = prerange.merge(GroupData,left_on=('GroupID'), right_on=('GroupID'))
    print(subrange.columns)
    print(f" Type {gt[1]} size {len(subrange)}")
    y = subrange[yfield]
    for i, xfield in enumerate(xfields) :
        x = np.log10(subrange[xfield]+0.01)
        xmin = min(x)
        xmax = max(x)
        X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
        positions = np.vstack([X.ravel(), Y.ravel()])
        #ax = fig.add_subplot(1, len(xfields), i)
        axes[i,j].set_title(gt[1])
        axes[i,j].set_ylabel(yfield)
        axes[i,j].set_xlabel('Mean(log10('+xfield+'))')
        values = np.vstack([x, y])
        kernel = stats.gaussian_kde(values)
        Z = np.reshape(kernel(positions).T, X.shape)
        #ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,extent=[xmin, xmax, ymin, ymax])
        #ax.imshow(np.rot90(Z), cmap='PuBu',extent=[xmin, xmax, ymin, ymax])
        #axes[i,j].imshow(np.rot90(Z), cmap='PuBu')
        #axes[i,j].set_xlim([xmax, xmax])
        #axes[i,j].set_ylim([-25, -19])
        #axes[0,2].set_xlim([0.7, 1.2])
        #axes[0,3].set_xlim([1.0, xmax])
        #axes[1,2].set_xlim([-0.5, 0])
        #axes[1,3].set_xlim([-0.4, -0.1])
        #axes[2,2].set_xlim([0.4,xmax])
        #axes[2,3].set_xlim([0.6,xmax])
        #axes[3,2].set_xlim([0,xmax])
        #axes[3,3].set_xlim([0.4,xmax])
        axes[i,j].contour(X, Y, Z)
        #axes[i,j].plot(x, y, 'k.', markersize=2)
                                
        #fig.legend(loc="upper right")
        #plt.axis([0, 3, 0, 0.8])
#fig.gca().invert_yaxis()
fig.tight_layout()
fig.savefig('../../ChartsPlots/GroupDensityTotRmagMeanLog10EnvironmentsGalaxyType.png', dpi=300, bbox_inches='tight')
plt.show()


GroupLogStar = GroupLogStar.to_pandas()
print(GroupLogStar.columns)

#def logsum(x) :
#    #return np.log10(np.sum(10**x))
#    return np.sum(x)

#GroupLogStar.assign(sumlog= GroupLogStar.groupby('GroupID')['logmstar'].agg(lambda x: np.log10(np.sum(10**x))))
#GroupLogStar['Sumlog']= GroupLogStar.groupby('GroupID')['logmstar'].agg(lambda x: np.log10(np.sum(10**x)))
#GroupLogStar['Sumlog']= GroupLogStar.groupby('GroupID')['logmstar'].agg(lambda x:np.sum(10**x))
#GroupLogStar['Sumlog']= GroupLogStar.groupby('GroupID')['logmstar'].agg(sum)
#GroupLogStar = GroupLogStar['GroupID','Sumlog']
#print(GroupLogStar.columns)

#GroupLogStar.reset_index(drop=True)
#GroupLogStar.set_index(['GroupID','Sumlog'])

#DataLocalGroup =
#GroupLogStar = GroupLogStar[['GroupID','Sumlog']]
#print(f"log min {min(GroupLogStar['Sumlog'])}")
#print(f"log max {max(GroupLogStar['Sumlog'])}")

GroupData = GroupLogStar.merge(GroupEnv, left_on=('GroupID'), right_on=('GroupID'))
print(GroupData.columns)
     
yfield = 'logmstar'

print(f"Min {min(GroupData[yfield])}")
print(f"Max {max(GroupData[yfield])}")

#fig = plt.figure(figsize=(10, 8), dpi=300)
fig, axes = plt.subplots(4,4,figsize=(12, 12), sharey=True) 
#fig.suptitle('Countour Plots - Group Redness versus Log(Environments) for Galaxy Type')

types = [(0,'Voids'),(1,'Sheet'),(2,'Filament'),(3,'Knot')]
xfields = ['CountInCyl_y','DistanceTo5nn_y','SurfaceDensity_y','AGEDenPar_y']
i = 0
j = 0

for j, gt in enumerate(types):
    prerange = GroupGal[GroupGal['GeoS4'] == gt[0]]
    subrange = prerange.merge(GroupData,left_on=('GroupID'), right_on=('GroupID'))
    print(subrange.columns)
    print(f" Type {gt[1]} size {len(subrange)}")
    y = subrange[yfield]
    for i, xfield in enumerate(xfields) :
        print(f"Min y {min(subrange[yfield])}")
        print(f"Max y {max(subrange[yfield])}")
        x = np.log10(subrange[xfield]+0.01)
        xmin = min(x)
        xmax = max(x)
        print(f"Min x {xmin}")
        print(f"Max x {xmax}")
        X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
        positions = np.vstack([X.ravel(), Y.ravel()])
        #ax = fig.add_subplot(1, len(xfields), i)
        axes[i,j].set_title(gt[1])
        axes[i,j].set_ylabel(yfield)
        axes[i,j].set_xlabel('Mean(log10('+xfield+'))')
        #sns.kdeplot(x=df.sepal_width, y=df.sepal_length)
        sns.kdeplot(x=subrange[xfield], y=subrange.logmstar,cmap='viridis',ax=axes[i,j])
        #values = np.vstack([x, y])
        #kernel = stats.gaussian_kde(values)
        #Z = np.reshape(kernel(positions).T, X.shape)
        #axes[i,j].contour(X, Y, Z)
    
        #plt.axis([0, 3, 0, 0.8])
#fig.gca().invert_yaxis()
fig.tight_layout()
fig.savefig('../../ChartsPlots/SumLogmstar_versus_mean_LogEnvironments_GalaxyType.png', dpi=300, bbox_inches='tight')
plt.show()
